### Video Capture and save the image on closing the video capture with blur effect


In [1]:
import cv2

cap = cv2.VideoCapture(0)

if not cap.isOpened():
    exit()
    
while True:
    ret, frame = cap.read()
    
    if not ret:
        break
    blurred_frame = cv2.GaussianBlur(frame, (45, 45), 0)
    cv2.imshow("Anvith", blurred_frame)
    
    
    if(cv2.waitKey(1) == ord('q')):
        cv2.imwrite("ME.png", frame)
        break

cap.release()
cv2.destroyAllWindows()
        


## Face Detection using Haar Cascade - Only cropped face within the window 


In [2]:
import cv2

def detect_face(img, faceCascade):
    grey_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(grey_image, 1.1, 5)
    
    if len(faces) == 0:
        return img
    
    for (x, y, w, h) in faces:
        cropped_face = img[y:y+h, x:x+w]
        return cropped_face
    
    return img

cap = cv2.VideoCapture(0)
faceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

while True:
    ret, img = cap.read()
    
    cropped_face = detect_face(img, faceClassifier)
    
    cv2.imshow("Cropped Face Detection", cropped_face)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


## Face Detection using Haar Cascade - Blurring background


In [3]:
a= 5 
print(a)

5


In [3]:
import cv2 
import numpy as np


def draw_boundary(img, classifier, scaleFactor, minNeighbours, color):
    
    #converting image into gray_scale
    grey_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    #to detect all the possible faces in the entire image
    features = classifier.detectMultiScale(grey_image, scaleFactor, minNeighbours)
    
    #to store the coordinates of the face
    coordinates = []
    
    for(x, y, w, h) in features:
        cv2.rectangle(img, (x,y), (x+w, y+h), color, 2)
        coordinates = [x, y, w, h]
    
    return coordinates, img


def detect_face(img, faceCascade):
    color = {"blue": (255, 0, 0), "green": (0, 255, 0), "red": (0,0,255)}
    
    coordinates, img = draw_boundary(img, faceCascade, 1.1, 5, color["red"])
    
    if len(coordinates) == 0:
        return img
    
    #masking the face
    mask = np.zeros_like(img)
    
    #getting the coordinates of the face
    x, y, w, h = coordinates
    
    #create a white rectangle for the face-mask
    cv2.rectangle(mask, (x,y), (x+w, y+h), (255,255,255), -1)
    
    #converting face region ti original color
    face_region = img[y:y+h, x:x+w]
    face_region = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)
    
    #applying mask to the face
    masked_face = cv2.bitwise_and(face_region, mask[y:y+h, x:x+w])
    
    #convert the face back to BGR
    masked_face = cv2.cvtColor(masked_face, cv2.COLOR_RGB2BGR)
    
    #blurring the background
    blurred = cv2.GaussianBlur(img, (99,99), 0)
    
    #replace face region in the blurred image
    blurred[y:y+h, x:x+w] = masked_face
    
    return blurred
    
        
        
    

cap = cv2.VideoCapture(0)

faceClassifer = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

while True:
    ret, img = cap.read()
    
    img = detect_face(img, faceClassifer)
    
    cv2.imshow("Face Detection", img)
    
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
cap.release()
cv2.destroyAllWindows()


## Measure Distance from the camera to the Detected Face 


In [3]:
import cv2
import numpy as np

# Constants for reference distance estimation
KNOWN_FACE_WIDTH = 16.5  # Width of the average face in centimeters
FOCAL_LENGTH = 650  # Focal length of the camera in pixels (adjust according to your camera)

def draw_boundary(img, classifier, scaleFactor, minNeighbours, color):
    # Converting image into grayscale
    gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    # Detecting all the possible faces in the entire image
    features = classifier.detectMultiScale(gray_image, scaleFactor, minNeighbours)

    # Storing the coordinates of the face
    coordinates = []

    for (x, y, w, h) in features:
        cv2.rectangle(img, (x, y), (x + w, y + h), color, 2)
        coordinates = [x, y, w, h]

    return coordinates, img


def detect_face(img, faceCascade):
    color = {"blue": (255, 0, 0), "green": (0, 255, 0), "red": (0, 0, 255)}

    coordinates, img = draw_boundary(img, faceCascade, 1.1, 5, color["red"])

    if len(coordinates) == 0:
        return img

    # Masking the face
    mask = np.zeros_like(img)

    # Getting the coordinates of the face
    x, y, w, h = coordinates

    # Create a white rectangle for the face-mask
    cv2.rectangle(mask, (x, y), (x + w, y + h), (255, 255, 255), -1)

    # Converting face region to original color
    face_region = img[y:y + h, x:x + w]
    face_region = cv2.cvtColor(face_region, cv2.COLOR_BGR2RGB)

    # Applying mask to the face
    masked_face = cv2.bitwise_and(face_region, mask[y:y + h, x:x + w])

    # Convert the face back to BGR
    masked_face = cv2.cvtColor(masked_face, cv2.COLOR_RGB2BGR)

    # Blurring the background
    blurred = cv2.GaussianBlur(img, (99, 99), 0)

    # Replace face region in the blurred image
    blurred[y:y + h, x:x + w] = masked_face

    # Calculate distance from the camera to the face
    distance = calculate_distance(w)

    # Display distance on the image
    cv2.putText(blurred, f"Distance: {distance:.2f} cm", (x, y - 10),
                cv2.FONT_HERSHEY_SIMPLEX, 0.7, (255, 255, 255), 2)

    return blurred


def calculate_distance(face_width):
    # Calculate the distance using the reference face width and focal length
    distance = (KNOWN_FACE_WIDTH * FOCAL_LENGTH) / face_width
    return distance


cap = cv2.VideoCapture(0)

faceClassifier = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

while True:
    ret, img = cap.read()

    img = detect_face(img, faceClassifier)

    cv2.imshow("Face Detection", img)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
